In [50]:
import pandas as pd
import numpy as np
from sklearn.metrics import log_loss
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
import pickle

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
plt.style.use('bmh')
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = 30, 20


import math
from scipy.stats import pearsonr, spearmanr, kendalltau, shapiro, pointbiserialr
from sklearn.linear_model import RandomizedLogisticRegression as RLR

In [51]:
train=pd.read_csv("train.csv")
train.columns

Index(['Months since Last Donation', 'Number of Donations',
       'Total Volume Donated (c.c.)', 'Months since First Donation',
       'Made Donation in March 2007'],
      dtype='object')

In [52]:
df=train

In [53]:
def createfeature(df):
    
    df['Average Donation per Month'] = (df['Total Volume Donated (c.c.)']/df['Months since First Donation'])
    df['Waiting Time'] = ((df['Months since First Donation'] - df['Months since Last Donation'])/df['Number of Donations'])
    #df['Donated in the past 3-6 months'] = ((df['Months since Last Donation'] >= 3) &(df['Months since Last Donation'] <= 6))
    #df['Frequent Donor'] = (df['Number of Donations'] >= 5)
    df['monthratio']=df['Months since Last Donation']/df['Months since First Donation']
    #df['monthratio']=df['Months since Last Donation']/df['Months since First Donation']
   
       
    df['Number of Donations sqrt']=np.sqrt(df['Number of Donations'].astype(float))
    df['Average Donation per Month sqrt']=np.sqrt(df['Average Donation per Month'])
    df['Months since First Donation sqrt']=np.sqrt(df['Months since First Donation'])
    df['Waiting Time sqrt']=np.sqrt(df['Waiting Time'])
    df['Months since Last Donation sqrt']=np.sqrt(df['Months since Last Donation'])

    df['Months since Last Donation mul']=df['Months since Last Donation']*df['Months since Last Donation']
    df['Number of Donations mul']=df['Number of Donations']*df['Number of Donations']
    df['Average Donation per Month mul']=df['Average Donation per Month']*df['Average Donation per Month']
    df['Waiting Time mul']=df['Waiting Time']*df['Waiting Time']
    df['Months since First Donation mul']=df['Months since First Donation']*df['Months since First Donation']
    
 

    means=df.mean(axis=1)

    df['mean']=means



    MAX=df.max(axis=1)

    df['max']=MAX


    MIN=df.min(axis=1)

    df['min']=MIN


    STD=df.std(axis=1)

    df['std']=STD


    MEDIAN=df.median(axis=1)

    df['median']=MEDIAN


    kurtosis=df.kurtosis(axis=1)

    df['kurtosis']=kurtosis


    skew=df.skew(axis=1)

    df['skew']=skew

    var=df.var(axis=1)

    df['var']=var

    return df

In [54]:
df=createfeature(df)
df.head(2)

,Months since Last Donation,Number of Donations,Total Volume Donated (c.c.),Months since First Donation,Made Donation in March 2007,Average Donation per Month,Waiting Time,monthratio,Number of Donations sqrt,Average Donation per Month sqrt,...,Waiting Time mul,Months since First Donation mul,mean,max,min,std,median,kurtosis,skew,var
0,2,50,12500,98,1,127.551020,1.920000,0.020408,7.071068,11.293849,...,3.686400,9604,2288.472495,16269.262807,0.020408,5564.961719,10.596672,2.061317,1.901873,2.706792e+07
1,0,13,3250,28,1,116.071429,2.153846,0.000000,3.605551,10.773645,...,4.639053,784,992.309953,13472.576531,0.000000,4034.003730,8.032574,6.946195,2.865862,1.410682e+07


In [55]:
df = df.drop(["Made Donation in March 2007","Total Volume Donated (c.c.)"],axis=1)

In [56]:
from sklearn.preprocessing import StandardScaler
#pd.options.mode.use_inf_as_na=True # treat inf as null


logdf=df.apply(np.log)
dfff=logdf.fillna(0)
dfff = dfff.replace([np.inf, -np.inf,np.nan], 0)

scaler = StandardScaler()
dff = pd.DataFrame(scaler.fit_transform(dfff))
dff.columns=dfff.columns

In [85]:
X_train, X_test, y_train, y_test = train_test_split(df, train['Made Donation in March 2007'], test_size=0.33, random_state=42)

#df_train, df_val = train_test_split(df, test_size=0.10,random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
print(X_train.__class__, X_test.__class__, y_train.__class__, y_test.__class__)

(385, 24) (191, 24) (385,) (191,)
<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'> <class 'pandas.core.series.Series'> <class 'pandas.core.series.Series'>


In [88]:
from sklearn.linear_model import LogisticRegressionCV
Classifier = [
  
        LGBMClassifier(
            
        n_estimators=132,#132
        learning_rate=0.1,
        num_leaves=64,
        max_depth=7,
        min_data_in_leaf=40,
        max_bin =15,
        reg_alpha=0.8, reg_lambda=0.6,
        colsample_bytree=1.0,
        min_split_gain=0.001, objective = "binary",random_state=42,
        stratified=True
                
        ),
        RandomForestClassifier(),
    
   
  
   
    ]

In [89]:
Accuracy=[]
Model=[]

# for classifier in Classifier:
#     try:
        
#         fit = classifier.fit(df_train[columns.tail(limit).index], df_train["Donation2007"])
#         pred = fit.predict_proba(df_val[columns.tail(limit).index])
#     except Exception:
#         fit = classifier.fit(df_train[columns.tail(limit).index], df_train["Donation2007"])
#         pred = fit.predict_proba(df_val[columns.tail(limit).index])
        
        
#     score = classifier.score(df_val[columns.tail(limit).index], df_val["Donation2007"])
#     Model.append(classifier.__class__.__name__)
#     print('Accuracy of '+classifier.__class__.__name__+' is '+str(score))
    
#     print("************************************************************")
#     val_actual=np.array(df_val["Donation2007"])
#     val_pred=np.array(pred)
    
    
for classifier in Classifier:
    try:
        
        fit = classifier.fit(X_train,y_train)
        pred = fit.predict(X_test)
    except Exception:
        fit = classifier.fit(X_train, y_train)
        pred = fit.predict(X_test)
        
        
    score = accuracy_score(y_test.values, pred)
    Model.append(classifier.__class__.__name__)
    print('Accuracy of '+classifier.__class__.__name__+' is '+str(score))
    
    print("************************************************************")
#     val_actual=np.array(df_val["Donation2007"])
#     val_pred=np.array(pred)

Accuracy of LGBMClassifier is 0.968586387435
************************************************************
Accuracy of RandomForestClassifier is 0.890052356021
************************************************************


In [94]:
lgbm = LGBMClassifier(   
        n_estimators=132,#132
        learning_rate=0.1,
        num_leaves=64,
        max_depth=7,
        min_data_in_leaf=40,
        max_bin =15,
        reg_alpha=0.8, reg_lambda=0.6,
        colsample_bytree=1.0,
        min_split_gain=0.001, objective = "binary",random_state=42,
        stratified=True
                 )
fit = lgbm.fit(X_train,y_train)
pickle.dump(fit,open( "blood_model.pickle", "wb" ) )

In [98]:
test=pd.read_csv("test.csv")
test=createfeature(test)
test = test.drop(["Total Volume Donated (c.c.)"],axis=1)

In [99]:
test.shape

(200, 24)

In [100]:
### without data scinece cycle follow

In [125]:
train=pd.read_csv("train.csv")
X_train, X_test, y_train, y_test = train_test_split(train[['Months since Last Donation', 'Number of Donations',
       'Total Volume Donated (c.c.)', 'Months since First Donation']], train['Made Donation in March 2007'], test_size=0.33, random_state=42)
lgbm = RandomForestClassifier()
fit = lgbm.fit(X_train,y_train)
pred = fit.predict(X_test)
accuracy_score(y_test.values, pred) 


0.72774869109947649